# Notebook #4: Cálculo de Rentabilidad

- Con los datos extraídos y el modelo de *machine learning* que se ha entrenado para predecir el precio del alquiler, somos capaces de realizar el cálculo de rentabilidad con ayuda de distintas funciones.

- Importamos las librerías y soportes.

In [432]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los dfFrames

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr
from src import soporte_yolo as sy
from src import soporte_scoring as ss
from src import soporte_blip as sb
from src import soporte_mongo as sm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- Como primer paso, definimos una variable con las rutas de los transformers: el encoder, el scaler y el modelo Random Forest.

- También vamos a conectarnos a la base de datos de Mongo para obtener el los datos correspondientes a viviendas en venta.

- Utilizaremos la función `predecir_alquiler` para aplicar el modelo a la totalidad de viviendas en venta, que devuelve el dataframe con una nueva columna 'alquiler_predicho'. Sus únicos argumentos son el archivo pickle con las viviendas en venta y la variable con las rutas de los transformers.

In [522]:
paths_transformers = ["../transformers/target_encoder.pkl", "../transformers/scaler.pkl", "../transformers/model.pkl"]
bd = sm.conectar_a_mongo("ProyectoRentabilidad")
gdf = sm.importar_a_geodataframe(bd, "venta")
gdf = sr.predecir_alquiler(gdf, paths_transformers)

- Comprobamos el dataframe.

- Para cada vivienda, tenemos una columna que corresponde a una lista con las urls de las imágenes. Para poder identificar cuál corresponde a cada estancia, utilizaremos el modelo YOLOv11, en su versión x y de clasificación, por cuanto no nos interesa determinar la ubicación ni la forma de los objetos en las imágenes, solamente su existencia.

- Con la función `identificar_urls_habitaciones`, se añaden etiquetas a la cocina y al baño de cada anuncio. Para optimizar el tiempo de procesamiento, cuando las ha encontrado, no continua con el resto de imágenes, sino que salta al siguiente anuncio.

- El resultado es un dataframe con nuevas columnas para la URL del baño y de la cocina. Cuando uno de los dos valores es nulo, elimina esa fila. El return también incluye un dataframe con todas las detecciones de las imágenes procesadas, como una forma de poder validar los resultados.

116 misses

In [523]:
gdf, detecciones = sy.identificar_urls_habitaciones(gdf, 'urls_imagenes', drop_nulls=True)

 33%|███▎      | 116/352 [36:30<1:06:39, 16.95s/it] 

Error processing https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/f3/c0/22/1263973405.webp: HTTPSConnectionPool(host='img4.idealista.com', port=443): Read timed out. (read timeout=10)
Error processing https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/a9/7b/cd/1263973421.webp: HTTPSConnectionPool(host='img4.idealista.com', port=443): Read timed out. (read timeout=10)


100%|██████████| 352/352 [1:13:56<00:00, 12.61s/it]

Se han eliminado 117 filas donde 'url_cocina' o 'url_banio' son None o NaN.


- Vamos a comprobar el resultado del filtrado.

- Además del dataframe con las imágenes filtradas, tenemos un segundo dataframe como retorno, que almacena los elementos que se han detectado en cada imagen, la URL y si ha sido etiquetado como cocina o baño. Esto nos permite realizar generar una métrica sobre la efectividad de YOLO.

- De las 365 viviendas en venta, se han encontrado imágenes de baño y cocina para 266. Dado que el cliente objetivo del proyecto no es quién se dedica a reformas, sin poder conocer el estado de estas estancias, el cálculo de rentabilidad puede cambiar significativamente por ser los espacios que cuesta más dinero reformar.

- Por ese motivo, las eliminaremos del conjunto de datos con el parámetro *drop nulls*.

In [525]:
gdf.shape[0]

265

In [526]:
detecciones[detecciones['habitación']=='kitchen'].head(5)

,url,detecciones,habitación
2,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, refrigerator, wardrobe, dishwasher...",kitchen
7,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, washbasin, refrigerator, washer, s...",kitchen
13,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[microwave, washer, stove, dishwasher, washbasin]",kitchen
14,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[washer, microwave, washbasin, refrigerator, s...",kitchen
16,https://img4.idealista.com/blur/WEB_LISTING-M/...,"[washer, microwave, space_heater, dishwasher, ...",kitchen


- Al comprobar las URLs de forma manual utilizando una muestra aleatoria, se confirma una accuracy de entorno al 90% en el etiquetado.

In [527]:
for value in detecciones[detecciones['habitación']=='kitchen']['url'].sample(10):
    print(value)

https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/69/26/e9/1278654596.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/d2/f1/46/1300762768.webp
https://img4.idealista.com/blur/WEB_LISTING-M/90/id.pro.es.image.master/e2/c2/66/1284302405.webp
https://img4.idealista.com/blur/WEB_LISTING-M/90/id.pro.es.image.master/27/0a/a8/1159363025.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/f9/90/42/1245503450.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/ff/30/25/1295857292.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/19/5a/4d/1300123246.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/7d/20/9c/1289004786.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/59/1a/b9/1300629581.webp
https://img4.idealista.com/blur/WEB_LISTING-M/0/id.pro.es.image.master/ca/f7/72/918499944.webp


- Como una capa de verificación adicional, vamos a utilizar el modelo BLIP de Salesforce, que crea descripciones de las imágenes, en este caso, de las URLs identificadas por YOLO. Contamos luego el número de ocurrencias de 'kitchen' o 'bathroom'.

In [528]:
captions_cocinas = sb.generar_descripciones(gdf, 'url_cocina')
sb.contar_palabras(captions_cocinas, 'kitchen')

265it [03:50,  1.15it/s]


233

In [510]:
captions_banios = sb.generar_descripciones(gdf, 'url_banio')
sb.contar_palabras(captions_banios, 'bathroom')

266it [04:24,  1.01it/s]


250

- De las 266 imágenes identificadas como cocinas por YOLO, BLIP ha coincidido en 234, un 88%. El dato en el caso de los baños mejora, habiendo encontrado 250 coincidencias, un 94%.

- Conocedores de que, habrá falsos negativos y falsos positivos en ambos modelos, el alto porcentaje de coincidencias en ambos casos nos indica que la fiabilidad del primer cribado es alta.

- Una observación adicional a lo detectado en el EDA es que, aquellos anuncios en que hemos rellenado el valor nulo del anunciante con 'ND', corresponden generalmente a particulares, lo cual es relevante porque podría significar entorno a un 3% de ahorro en el valor de compra.

In [529]:
gdf[gdf['anunciante']=='ND'].sample(2)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,distrito,geometry,alquiler_predicho,url_cocina,url_banio
234,102216234,144900.0,1932.0,piso,True,3,False,75.0,4,1,False,False,True,False,good,"calle Portugal, 16","Piso ubicado en la zona universitaria, en CALL...",ND,652561233,['https://img4.idealista.com/blur/WEB_LISTING-...,Delicias,POINT (-0.90189 41.65036),913.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
66,106918117,134000.0,2436.0,piso,True,4,False,55.0,2,1,True,False,False,False,good,"calle del Río Duero, 30","Esta encantadora y acogedora vivienda, con un ...",ND,ND,['https://img4.idealista.com/blur/WEB_LISTING-...,La Almozara,POINT (-0.89903 41.65904),686.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...


- El siguiente paso será utilizar la API de Anthropic, y específicamente el modelo Sonnet, para asignar una puntuación del 1 al 10 al estado de la estancia, y una estimación de los metros cuadrados, que es una consideración que ha surgido durante el desarrollo y la fase de pruebas del proyecto.

- El prompt de sistema es similar a este:

<div style="background-color:rgb(149, 222, 237); padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">
You are an AI image analysis system specialized in evaluating property conditions. Your task is to analyze multiple property images in batch and provide a precise evaluation.
Instructions for Analysis:

Analyze each pair of images:

Image 1: Kitchen
Image 2: Bathroom

Evaluation Criteria:

Ratings: Whole numbers 1-5
1: Very poor (complete renovation required)
2: Poor (major renovations needed)
3: Fair (some renovations needed)
4: Good (minor improvements required)
5: Excellent (no renovations needed)

Sizes: Whole numbers in square meters (m²)
</div>

- Como puede verse en el prompt, el resultado de la consulta son 4 valores separados por comas, con la función `analizar_propiedades`, separaremos esos resultados y los asignaremos a 4 nuevas columnas del dataframe. Para tener consultas más eficientes, se enviarán en lotes, o *batches*, que corresponden a grupos de viviendas. Este batch más el dataframe con las URLs identificadas son los parámetros de la función.

In [530]:
df_sample = gdf.head(3)

In [532]:
df_sample, resultados = ss.analizar_propiedades(df_sample, batch=3)

Procesando lotes:   0%|          | 0/1 [00:00<?, ?it/s]

In [533]:
df_sample

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,distrito,geometry,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.86935 41.6973),1180.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,5,20,5
1,106086132,139900.0,1686.0,piso,True,3,True,83.0,2,1,False,False,True,False,good,calle de Portugal,Financiación hipoteca 100% ¡oportunidad única ...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,Delicias,POINT (-0.90232 41.65202),761.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,18,6
2,105844791,149900.0,1162.0,piso,True,4,True,129.0,3,2,False,False,False,False,good,carretera de Huesca,Financiación hipoteca 100%. ¡oportunidad única...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.87 41.69848),1186.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,16,4


In [547]:
df_sample.to_pickle("../streamlit/ejemplo.pkl")

In [334]:
gdf.to_pickle("../data/transformed/final_tags.pkl")

In [335]:
df_importado = pd.read_pickle("../data/transformed/final_tags.pkl")
df_importado.shape[0]

278

In [536]:
df_ej = sr.calcular_rentabilidad_inmobiliaria_wrapper(df_sample, porcentaje_entrada=0.2, coste_reformas=10000, comision_agencia=5000, anios=30, tin=0.03, seguro_vida=0, tipo_irpf=0.37, porcentaje_amortizacion=0.4)
df_ej.to_csv("../ejemplo.csv")

In [538]:
df_ej

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,distrito,geometry,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio,Coste Total,Rentabilidad Bruta,Beneficio Antes de Impuestos,Rentabilidad Neta,Cuota Mensual Hipoteca,Cash Necesario Compra,Cash Total Compra y Reforma,Beneficio Neto,Cashflow Antes de Impuestos,Cashflow Después de Impuestos,ROCE,ROCE (Años),Cash-on-Cash Return,COCR (Años)
2,105844791,149900.0,1162.0,piso,True,4,True,129.0,3,2,False,False,False,False,good,carretera de Huesca,Financiación hipoteca 100%. ¡oportunidad única...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.87 41.69848),1186.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,16,4,179890.0,7.91,8392.27,3.96,-505.59,49970.0,54970.0,7128.25,4394.93,3130.92,23.73,4.21,5.22,19.15
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,Distrito Rural,POINT (-0.86935 41.6973),1180.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,5,20,5,179890.0,7.87,8333.95,3.94,-505.59,49970.0,54970.0,7082.88,4336.61,3085.54,23.61,4.24,5.15,19.44
1,106086132,139900.0,1686.0,piso,True,3,True,83.0,2,1,False,False,True,False,good,calle de Portugal,Financiación hipoteca 100% ¡oportunidad única ...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,Delicias,POINT (-0.90232 41.65202),761.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,18,6,168890.0,5.41,4439.81,2.38,-471.86,46970.0,51970.0,4019.94,709.14,289.27,16.03,6.24,0.51,196.94
